In [ ]:
# -*- coding:utf-8 -*-
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import datetime
import random

In [ ]:
#중간 저장 주기 = x회 마다
save_when = 100

#파일 경로
save_path = 'c:/cr/'
font_path_window="c:/Windows/Fonts/malgun.ttf"
driver_path = 'c:/cd/chromedriver'

#파일 형식
save_type = 'csv'

#다음페이지 넘어갈 때 기다리는 시간(최소, 최대)
wait_time = random.uniform(1.5, 2.5)

#검색 기준: 인기(hot), 최신(new)
find_index = 'hot'

In [3]:
browser = webdriver.Chrome(driver_path)
browser.implicitly_wait(3)

In [4]:
def searching_insta(browser):
    url = 'https://www.instagram.com/explore/tags/{}'.format(word)
    browser.get(url)

In [ ]:
def click_article(browser, index):
    if index == 'hot':
        browser.find_element_by_css_selector('a > div > div._si7dy').click()
    
    elif index == 'new':
        browser.find_element_by_css_selector('div._cmdpi > div > div').click()

In [6]:
def get_current_url(browser):
    article_url = browser.current_url
    return article_url

In [7]:
def get_contents(browser):
    author = browser.find_element_by_css_selector('div._j56ec > div > div > a').text
    contents = [author]
    
#     #첫번째 댓글만 가져오기
#     article = browser.find_element_by_css_selector('li._ezgzd > span').text
    
#     #더보기를 누르지 않은 상태에서 모든 댓글을 가져오기
#     article = browser.find_element_by_css_selector('div._ebcx9 > div._4a48i._277v9 > ul').text
    
    #더보기를 눌러서 모든 댓글을 가져오기
    more_reply = True
    while more_reply:
        try:
            browser.find_element_by_css_selector('a._m3m1c._1s3cd').click()
            browser.implicitly_wait(1)
            
        except:
            more_reply = False
            break
    
    replies = browser.find_elements_by_css_selector('li._ezgzd')
    article = []
    for reply in replies:
        reply_author = reply.find_element_by_css_selector('a').text
        reply_article = reply.find_element_by_css_selector('span').text
        article.append([reply_author, reply_article])

    contents.append(article)
        
    return contents

In [8]:
def get_content_type(browser):
    try:
        video_check = browser.find_element_by_css_selector('section._1w76c._nlmjy > div > span')
        if video_check.text[:2] == '조회':
            content_type = '동영상'
            
        else:
            content_type = '이미지'
    
    except:
        content_type = '이미지'
        
    return content_type

In [9]:
def get_img_likes(browser):
    plays = 'none'
    
    try:      
        check = browser.find_element_by_css_selector('section._1w76c._nlmjy > div').text
                
        if check[-3:] == '합니다':
            likes = len(browser.find_elements_by_css_selector('section._1w76c._nlmjy > div > a'))
                        
        else:
            likes = int(check[4:-1].replace(',',''))
                                
        return likes, plays
                                                           
    except:
        likes = 0
        return likes, plays

In [10]:
def get_vid_likes(browser):
    check = browser.find_element_by_css_selector('section._1w76c._nlmjy > div > span > span')

    plays = check.text
           
    check.click()
        
    likes = browser.find_element_by_css_selector('section._1w76c._nlmjy > div > div > div._m10kk > span').text
        
    browser.find_element_by_css_selector('section._1w76c._nlmjy > div > div > div._7px37').click()
    
    plays = int(plays.replace(',',''))
    likes = int(likes.replace(',',''))
    
    return likes, plays

In [11]:
def get_likesandplays(browser):
    content_type = get_content_type(browser)
    if content_type == '이미지':
        likesandplays = get_img_likes(browser)
    
    elif content_type == '동영상':
        likesandplays = get_vid_likes(browser)
    
    return likesandplays

In [12]:
def get_location(browser):
    try:
        location = browser.find_element_by_css_selector('div._j56ec > div._60iqg > a').text
    
    except NoSuchElementException:
        location = 'none'
    
    return location

In [13]:
def move_next(browser, wait_time):
    try:
        browser.find_element_by_css_selector('div._23gmb > div > div > a._3a693.coreSpriteRightPaginationArrow').click()
        time.sleep(wait_time)
        
    except NoSuchElementException:
        print('마지막 게시물입니다.')

In [14]:
def save():
    txt_path = save_path + '{}_{}_{}.txt'.format(word, start_time, save_count)
    csv_path = save_path + '{}_{}_{}.csv'.format(word, start_time, save_count)
    
    if save_type == 'csv':      
        report = pd.DataFrame(results, columns = ['좋아요', '재생수', '작성자', '장소', '본문', 'url'])
        report.to_csv(csv_path)
    
    elif save_type == 'txt':
        f = open(txt_path, 'w+', encoding = 'utf8')
        f.write('\n'.join(results))
        f.close()
       
    save_num = save_count + 1
    return save_num

In [15]:
word = input('찾고 싶은 키워드를 입력하세요: ')
searching_insta(browser)

num = int(input('몇건의 포스팅을 찾으시겠습니까?: '))

click_article(browser, find_index)
browser.implicitly_wait(3)

save_count = 1

now = datetime.datetime.now()
start_time = now.strftime('%Y%m%d_%H%M')

results = []

for i in range(num):                     
    index = int(i+1)
    print('{}번째 크롤링 중'.format(index))

    likes = get_likesandplays(browser)[0]
    plays = get_likesandplays(browser)[1]
    author = get_contents(browser)[0]
    location = get_location(browser)
    article = get_contents(browser)[1]
    article_url = get_current_url(browser)

    result = [likes, plays, author, location, article, article_url]
    results.append(result)   

    if index % save_when == 0:
        save_count = save(save_count)
        print('- 중간 결과를 저장했습니다 -')

    try:
        move_next(browser, wait_time)
        
    except NoSuchElementException:
        print('마지막 게시물입니다')
        break

save_count = save()

print('''
- 최종 결과를 저장했습니다 -
- 모든 크롤링을 마쳤습니다 - 
''')

찾고 싶은 키워드를 입력하세요: 펫푸드
몇건의 포스팅을 찾으시겠습니까?: 200
1번째 크롤링 중
2번째 크롤링 중
3번째 크롤링 중
4번째 크롤링 중
5번째 크롤링 중
6번째 크롤링 중
7번째 크롤링 중
8번째 크롤링 중
9번째 크롤링 중
10번째 크롤링 중
11번째 크롤링 중
12번째 크롤링 중
13번째 크롤링 중
14번째 크롤링 중
15번째 크롤링 중
16번째 크롤링 중
17번째 크롤링 중
18번째 크롤링 중
19번째 크롤링 중
20번째 크롤링 중
21번째 크롤링 중
22번째 크롤링 중
23번째 크롤링 중
24번째 크롤링 중
25번째 크롤링 중
26번째 크롤링 중
27번째 크롤링 중
28번째 크롤링 중
29번째 크롤링 중
30번째 크롤링 중
31번째 크롤링 중
32번째 크롤링 중
33번째 크롤링 중
34번째 크롤링 중
35번째 크롤링 중
36번째 크롤링 중
37번째 크롤링 중
38번째 크롤링 중
39번째 크롤링 중
40번째 크롤링 중
41번째 크롤링 중
42번째 크롤링 중
43번째 크롤링 중
44번째 크롤링 중
45번째 크롤링 중
46번째 크롤링 중
47번째 크롤링 중
48번째 크롤링 중
49번째 크롤링 중
50번째 크롤링 중
- 중간 결과를 저장했습니다 -
51번째 크롤링 중
52번째 크롤링 중
53번째 크롤링 중
54번째 크롤링 중
55번째 크롤링 중
56번째 크롤링 중
57번째 크롤링 중
58번째 크롤링 중
59번째 크롤링 중
60번째 크롤링 중
61번째 크롤링 중
62번째 크롤링 중
63번째 크롤링 중
64번째 크롤링 중
65번째 크롤링 중
66번째 크롤링 중
67번째 크롤링 중
68번째 크롤링 중
69번째 크롤링 중
70번째 크롤링 중
71번째 크롤링 중
72번째 크롤링 중
73번째 크롤링 중
74번째 크롤링 중
75번째 크롤링 중
76번째 크롤링 중
77번째 크롤링 중
78번째 크롤링 중
79번째 크롤링 중
80번째 크롤링 중
81번째 크롤링 중
82번째 크롤링 중
83번째 크롤링 중
84번째 크롤링 중
85번째 크롤링 중
86번째 크롤링 중
